In [3]:
"""@property装饰器

Python内置的@property装饰器
负责把一个方法变成属性调用，简化对属性的getter/setter操作
"""

class Student:
    
    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 and 100')
        self._score = value
        

s = Student()
s.score = 99
print(s.score)

s.score = 101

99


ValueError: score must between 0 and 100

In [4]:
# 使用@property装饰器创建动态（只读）属性
# 动态（只读）属性不会被实际存储，而是在需要的时候被计算出来

import math

class Circle:
    
    def __init__(self, r):
        self.r = r
    
    @property
    def diameter(self):
        return self.r * 2
    
    @property
    def perimeter(self):
        return 2 * math.pi * self.r
    
    @property
    def area(self):
        return math.pi * self.r ** 2
    
    
# 测试
c = Circle(3)
print('半径：', c.r)
print('直径：', c.diameter)
print('周长：', c.perimeter)
print('面积：', c.area)

半径： 3
直径： 6
周长： 18.84955592153876
面积： 28.274333882308138


In [5]:
"""装饰器

装饰器类可以实现大部分类特性中的底层魔术方法，包括@classmethod @staticmethod @property，甚至__slots__属性
通过定义一个装饰器类，可以在底层捕获核心的实例操作（get、set、delete），并且可完全自定义它们的行为
装饰器只能在类级别被定义，而不能为每个实例单独定义
定义一个延迟计算属性的高效方法是通过使用装饰器类
"""

# 自定义装饰器类
class lazyproperty:
    
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value

        
# 在其他类中使用描述器类
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    @lazyproperty
    def diameter(self):
        print('计算直径：')
        return self.radius * 2
    
    @lazyproperty
    def perimeter(self):
        print('计算周长：')
        return 2 * math.pi * self.radius
    
    @lazyproperty
    def area(self):
        print('计算面积：')
        return math.pi * self.radius ** 2
    
    
# 测试
c = Circle(3)
print(c.diameter)
print(c.perimeter)
print(c.area)

计算直径：
6
计算周长：
18.84955592153876
计算面积：
28.274333882308138


In [6]:
# 在类中，一个property其实是getter、setter和deleter方法的集合，而不是单个方法
# 父类
class SuperPerson:
    
    def __init__(self, name):
        self.name = name
        
    # Getter
    @property
    def name(self):
        return self._name
    
    # Setter
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value
        
    # Deleter
    @name.deleter
    def name(self):
        raise AttributeError('Can\'t delete attribute')
        

# 在子类中扩展property
# 当扩展一个property的时候，需要先确定是否要重新定义所有的方法还是只修改其中某一个
# 子类
class SubPerson(SuperPerson):
    
    # Getter
    @property
    def name(self):
        print('Getting name')
        return super().name
    
    # Setter
    @name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)
    
    # Deleter
    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)
        

# 测试
person = SubPerson('Jordan')
print(person.name)
person.name = 'James'
print(person.name)

Setting name to Jordan
Getting name
Jordan
Setting name to James
Getting name
James


In [18]:
# 改变对象实例的打印或显示输出，使其更有可读性
# 重新定义类的__repr__和__str__方法
# __repr__方法返回实例的代码表示，通常用来重新构造这个实例，可以在使用交互式解析器（如Jupyter Notebook）时显示
# __str__方法将实例转换为一个字符串，使用str()和print()函数会输出这个字符串


class Circle:
    
    def __init__(self, r):
        self.r = r
    
    def __repr__(self):
        return 'Circle({0.r})'.format(self)
    
    def __str__(self):
        return 'This is Circle R{0.r}'.format(self)


# 测试    
p = Circle(3)
print(p)

# !r格式化指明使用__repr__来代替默认的__str__
print('{0!r}'.format(p))

# 交互式解释器输出
p

This is Circle R3
Circle(3)


Circle(3)

In [25]:
# 通过format()函数和字符串方法使对象支持自定义的格式化

# 集合
_formats = {
    'ymd': '{d.year}-{d.month}-{d.day}',
    'mdy': '{d.month}/{d.day}/{d.year}',
    'dmy': '{d.day}/{d.month}/{d.year}'
}


class Fdate:
    
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
        
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d = self)
    
    
# 测试
fd = Fdate(2020, 4, 24)
print(format(fd))
print(format(fd, 'mdy'))
print('Today is {:dmy}'.format(fd))

2020-4-24
4/24/2020
Today is 24/4/2020
